In [2]:
import os
os.environ['TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL'] = '1'  # Для AMD GPU
import torch
print(torch.__version__)  # Должно вывести: 2.5.1+rocm6.2
print(torch.cuda.is_available())  # Проверка работы ROCm (True)

2.5.1+rocm6.2
True


In [6]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from ruwordnet import RuWordNet
import re
import numpy as np
import faiss
import numpy as np
from typing import List, Dict, Any

In [5]:
# Инициализация компонентов
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
morph_vocab = MorphVocab()
# Инициализация RuWordNet
wn = RuWordNet()  
sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
class TextPreprocessor:
    """
    Класс для предобработки текста: очистка, лемматизация, генерация эмбеддингов.
    """
    
    def __init__(self, use_synonyms=True):
        # Инициализация компонентов Natasha
        self.segmenter = segmenter
        self.morph_tagger = morph_tagger
        self.syntax_parser = syntax_parser
        self.morph_vocab = morph_vocab
        
        # Настройки для синонимов
        self.use_synonyms = use_synonyms
        self.wn = wn if use_synonyms else None
        
        # Кэш для ускорения повторных обработок
        self.lemma_cache = {}
        self.synonym_cache = {}
    
    def clean_text(self, text):
        """Очистка текста от лишних символов и нормализация"""
        # Приведение к нижнему регистру и замена ё
        text = text.lower().replace("ё", "е")
        
        # Удаление URL-ссылок
        text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
        
        # Удаление эмодзи и специальных символов
        emoji_pattern = re.compile(
            '['
            u'\U0001F600-\U0001F64F'  # эмоции
            u'\U0001F300-\U0001F5FF'  # символы
            u'\U0001F680-\U0001F6FF'  # транспорт
            u'\U0001F700-\U0001F77F'  # алхимия
            u'\U0001F780-\U0001F7FF'  # геометрические фигуры
            u'\U0001F800-\U0001F8FF'  # дополнительные символы
            u'\U0001F900-\U0001F9FF'  # дополнительные символы-2
            u'\U0001FA00-\U0001FA6F'  # шахматы
            u'\U0001FA70-\U0001FAFF'  # дополнительные символы-3
            u'\U00002702-\U000027B0'  # Dingbats
            u'\U000024C2-\U0001F251'  # Enclosed
            ']+', 
            flags=re.UNICODE
        )
        text = emoji_pattern.sub(' ', text)
        
        # Удаление HTML-сущностей и специальных символов
        text = re.sub(r'&[a-z]+;', ' ', text)
        
        # Удаление пунктуации, цифр и не-буквенных символов
        text = re.sub(r'[^a-zа-я\s]', ' ', text)
        
        # Удаление телеграм-упоминаний и бот-команд
        text = re.sub(r'@\w+|/\w+', ' ', text)
        
        # Удаление лишних пробелов и обрезка
        return re.sub(r'\s+', ' ', text).strip()
    
    def lemmatize(self, text):
        """Лемматизация текста с помощью Natasha"""
        # Проверка кэша
        if text in self.lemma_cache:
            return self.lemma_cache[text]
            
        # Обработка с Natasha
        doc = Doc(text)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        
        # Получение лемм
        lemmas = []
        for token in doc.tokens:
            if token.pos != 'PUNCT':
                token.lemmatize(self.morph_vocab)
                lemmas.append(token.lemma)
        
        self.lemma_cache[text] = lemmas
        return lemmas
    
    def get_synonyms(self, word, max_synonyms=3):
        """Получение синонимов из RuWordNet"""
        if not self.use_synonyms:
            return []
            
        # Проверка кэша
        cache_key = f"{word}_{max_synonyms}"
        if cache_key in self.synonym_cache:
            return self.synonym_cache[cache_key]
            
        try:
            synsets = self.wn.get_synsets(word)
            synonyms = []
            
            if synsets:
                for synset in synsets[:2]:
                    for sense in synset.senses:
                        if sense.word != word and sense.word not in synonyms:
                            synonyms.append(sense.word)
                            if len(synonyms) >= max_synonyms:
                                break
                    if len(synonyms) >= max_synonyms:
                        break
                        
            self.synonym_cache[cache_key] = synonyms
            return synonyms
        except Exception:
            return []
    
    def create_bigrams(self, tokens):
        """Создание биграмм из токенов"""
        if len(tokens) < 2:
            return []
        return [' '.join(tokens[i:i+2]) for i in range(len(tokens)-1)]
    
    def process_document(self, text):
        """
        Полная обработка документа с сохранением позиций токенов.
        Возвращает структуру данных с информацией для индексации.
        """
        # Очистка и лемматизация
        clean_text = self.clean_text(text)
        lemmas = self.lemmatize(clean_text)
        
        # Создание биграмм
        bigrams = self.create_bigrams(lemmas)
        
        # Формирование результата с сохранением информации о позициях
        tokens = []
        token_positions = []
        token_types = []  # 'unigram' или 'bigram'
        
        # Добавление униграмм
        for i, lemma in enumerate(lemmas):
            tokens.append(lemma)
            token_positions.append(i)
            token_types.append('unigram')
        
        # Добавление биграмм
        for i, bigram in enumerate(bigrams):
            tokens.append(bigram)
            token_positions.append(i)  # Позиция начала биграммы
            token_types.append('bigram')
        
        return {
            'original_text': text,
            'clean_text': clean_text,
            'lemmas': lemmas,
            'tokens': tokens,
            'token_positions': token_positions,
            'token_types': token_types
        }
    
    def process_query(self, text):
        """
        Обработка поискового запроса с расширением синонимами.
        """
        clean_text = self.clean_text(text)
        lemmas = self.lemmatize(clean_text)
        
        # Расширение запроса синонимами
        expanded_terms = lemmas.copy()
        if self.use_synonyms:
            for lemma in lemmas:
                synonyms = self.get_synonyms(lemma)
                expanded_terms.extend(synonyms)
        
        return {
            'original_query': text,
            'clean_query': clean_text,
            'lemmas': lemmas,
            'expanded_terms': expanded_terms
        }


In [27]:
preprocessor = TextPreprocessor(use_synonyms=True)


In [28]:
txt1 = "Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈"
clean1 = preprocessor.process_document(txt1)
print(clean1.get('original_text'))
print(clean1.get('clean_text'))
print(clean1.get('lemmas')[0:10])

Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈
твой лучший секс спрятан здесь делюсь каналом дипломированного сексолога крис взломала код классного секса мастерски раскрепощает знает миллион горячих техник и лучшие девайсы для взрослых самые полезные посты здесь отрезвляющий пост я все сама прокачай наездницу ролевая игра vip кинотеатр техника оральных ласк как занимается сeксом неудобная женщина кстати крис провела трехдневный безопла

In [29]:
query1 = preprocessor.process_query('конфеты')
print(query1.get('original_query'))
print(query1.get('clean_query'))
print(query1.get('lemmas'))
print(query1.get('expanded_terms'))

конфеты
конфеты
['конфета']
['конфета']


In [36]:
preprocessor.get_synonyms('красивый')

Synset(id="114650-A", title="БЛАГОЗВУЧНЫЙ") ['БЛАГОЗВУЧНЫЙ', 'КРАСИВЫЙ', 'ПРИЯТНЫЙ НА СЛУХ', 'КРАСИВЫЙ НА СЛУХ']


[]

In [31]:
wn.get_synsets('красивый')

[Synset(id="115077-A", title="КРАСИВЫЙ НА ВИД"),
 Synset(id="119209-A", title="НРАВСТВЕННЫЙ, ЭТИЧНЫЙ"),
 Synset(id="114650-A", title="БЛАГОЗВУЧНЫЙ")]

In [33]:
for sense in wn.get_senses('замок'):
    print(sense.synset)

Synset(id="126228-N", title="СРЕДНЕВЕКОВЫЙ ЗАМОК")
Synset(id="114707-N", title="ЗАМОК ДЛЯ ЗАПИРАНИЯ")


In [34]:
for sense in wn.get_senses('собака'):
    print(sense.synset, [synonym.name for synonym in sense.synset.senses])

Synset(id="4454-N", title="СОБАКА") ['СОБАКА', 'ПЕС', 'СОБАЧКА', 'СОБАЧОНКА', 'ПСИНА', 'ЧЕТВЕРОНОГИЙ ДРУГ', 'ПЕСИК']


In [35]:
for sense in wn.get_senses('красивый'):
    print(sense.synset, [synonym.name for synonym in sense.synset.senses])

Synset(id="115077-A", title="КРАСИВЫЙ НА ВИД") ['КРАСИВЫЙ', 'ЭСТЕТИЧНЫЙ', 'КРАСИВЫЙ НА ВИД', 'ВНЕШНЕ КРАСИВЫЙ', 'КРАСИВЫЙ ВНЕШНЕ', 'КРАСИВЕЙШИЙ']
Synset(id="119209-A", title="НРАВСТВЕННЫЙ, ЭТИЧНЫЙ") ['МОРАЛЬНЫЙ', 'КРАСИВЫЙ', 'ЧИСТЫЙ', 'НРАВСТВЕННЫЙ', 'ЭТИЧНЫЙ', 'ВЫСОКОНРАВСТВЕННЫЙ', 'ЧИСТЕЙШИЙ', 'ВЫСОКОМОРАЛЬНЫЙ', 'ВЫСОКОПОРЯДОЧНЫЙ', 'НРАВСТВЕННО ЧИСТЫЙ', 'НЕЗАМУТНЕННЫЙ', 'ЧИСТЕНЬКИЙ']
Synset(id="114650-A", title="БЛАГОЗВУЧНЫЙ") ['БЛАГОЗВУЧНЫЙ', 'КРАСИВЫЙ', 'ПРИЯТНЫЙ НА СЛУХ', 'КРАСИВЫЙ НА СЛУХ']


In [37]:
def get_synonyms__(word):
    synonyms = []
    for synset in wn.get_synsets(word):
        for lemma in synset.lemmas():
            synonyms.append(lemma.name())
    return synonyms


In [40]:
synonyms = [synonym.name() for synset in wn.get_synsets("ворона") for synonym in synset.lemmas()]


AttributeError: 'Synset' object has no attribute 'lemmas'

In [13]:
class FaissVectorStore:
    """
    Класс для хранения и поиска векторов с использованием FAISS.
    """
    
    def __init__(self, model=None, embedding_dim=None):
        """
        Инициализация хранилища векторов.
        
        Args:
            model: модель для определения размерности эмбеддингов
            embedding_dim: явно указанная размерность эмбеддингов
        """
        # Если размерность не указана явно, определим с помощью модели
        if embedding_dim is None and model is not None:
            # Получаем размерность из модели, создав тестовый эмбеддинг
            test_embedding = model.encode("тестовый текст")
            self.embedding_dim = test_embedding.shape[0]
        else:
            self.embedding_dim = embedding_dim or 768  # По умолчанию для BERT
            
        # Создание индекса FAISS
        self.index = faiss.IndexFlatIP(self.embedding_dim)
        self.documents = []
        self.token_metadata = []
    
    def add_document(self, doc_data, model):
        """
        Добавление документа в индекс.
        
        Args:
            doc_data: обработанные данные документа
            model: модель для создания эмбеддингов
        """
        # Сохраняем документ
        doc_id = len(self.documents)
        self.documents.append({
            'id': doc_id,
            'original_text': doc_data['original_text'],
            'clean_text': doc_data['clean_text'],
            'lemmas': doc_data['lemmas']
        })
        
        # Создаем эмбеддинги для каждого токена
        tokens = doc_data['tokens']
        if not tokens:
            return
        
        # Получаем эмбеддинги    
        embeddings = model.encode(tokens)
        
        # Обеспечиваем правильную форму для одиночных эмбеддингов
        if isinstance(embeddings, list):
            embeddings = np.array(embeddings)
        if len(embeddings.shape) == 1:
            embeddings = embeddings.reshape(1, -1)
            
        # Проверяем размерность
        if embeddings.shape[1] != self.embedding_dim:
            raise ValueError(f"Размерность эмбеддингов ({embeddings.shape[1]}) не соответствует размерности индекса ({self.embedding_dim})")
        
        # Добавляем метаданные токенов
        for i, token in enumerate(tokens):
            self.token_metadata.append({
                'doc_id': doc_id,
                'token': token,
                'position': doc_data['token_positions'][i],
                'type': doc_data['token_types'][i]
            })
        
        # Добавляем эмбеддинги в индекс FAISS
        self.index.add(embeddings.astype(np.float32))



In [9]:
class SemanticSearcher:
    """
    Класс для семантического поиска текста с использованием FAISS.
    """
    
    def __init__(self, preprocessor: TextPreprocessor, vector_store: FaissVectorStore, model):
        """
        Инициализация поисковика.
        
        Args:
            preprocessor: класс для предобработки текста
            vector_store: хранилище векторов
            model: модель для эмбеддингов
        """
        self.preprocessor = preprocessor
        self.vector_store = vector_store
        self.model = model
    
    def search(self, query: str, k: int = 5, threshold: float = 0.7):
        """
        Выполнение поиска по запросу.
        
        Args:
            query: поисковый запрос
            k: количество результатов
            threshold: порог сходства
            
        Returns:
            list: найденные совпадения
        """
        # Обработка запроса
        processed_query = self.preprocessor.process_query(query)
        
        # Получение эмбеддинга запроса
        query_embedding = self.model.encode(processed_query['clean_query'])
        if len(query_embedding.shape) == 1:
            query_embedding = query_embedding.reshape(1, -1)
        
        # Поиск ближайших векторов в индексе
        D, I = self.vector_store.index.search(query_embedding.astype(np.float32), k=min(k, self.vector_store.index.ntotal))
        
        # Формирование результатов
        results = []
        for i, (score, idx) in enumerate(zip(D[0], I[0])):
            if score < threshold:
                continue
                
            # Получение метаданных
            metadata = self.vector_store.token_metadata[idx]
            doc_id = metadata['doc_id']
            document = self.vector_store.documents[doc_id]
            
            # Формирование результата
            result = {
                'document': document['original_text'],
                'token': metadata['token'],
                'position': metadata['position'],
                'confidence': float(score),
                'is_bigram': metadata['type'] == 'bigram',
                'doc_id': doc_id
            }
            
            results.append(result)
        
        # Сортировка по убыванию уверенности
        results.sort(key=lambda x: x['confidence'], reverse=True)
        
        return results


In [15]:
# Инициализация классов
preprocessor = TextPreprocessor(use_synonyms=True)

# Сначала создаем модель
# sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')

# Затем создаем хранилище векторов, передавая модель для определения размерности
vector_store = FaissVectorStore(model=sbert_model)

# Создаем поисковик
searcher = SemanticSearcher(preprocessor, vector_store, sbert_model)

# Добавление документов в индекс
documents = [
    "он продал свой портрет",
    "он вообще не собирается переезжать в другое государство"
]

# Обработка и индексация документов
for document in documents:
    doc_data = preprocessor.process_document(document)
    vector_store.add_document(doc_data, sbert_model)

# Сохранение индекса
vector_store.save("semantic_search_index")


AttributeError: 'FaissVectorStore' object has no attribute 'save'

In [14]:
# Инициализация классов
preprocessor = TextPreprocessor(use_synonyms=True)
vector_store = FaissVectorStore(embedding_dim=768)  # Размерность эмбеддингов SBERT
searcher = SemanticSearcher(preprocessor, vector_store, sbert_model)

# Добавление документов в индекс
documents = [
    "он продал свой портрет",
    "он вообще не собирается переезжать в другое государство"
]

# Обработка и индексация документов
for document in documents:
    doc_data = preprocessor.process_document(document)
    vector_store.add_document(doc_data, sbert_model)

# Сохранение индекса
vector_store.save("semantic_search_index")

# Поиск по запросам
results1 = searcher.search("картина", k=3)
results2 = searcher.search("страна", k=3)

# Вывод результатов
print("Результаты для запроса 'картина':")
for r in results1:
    print(f"Найдено '{r['token']}' в документе '{r['document']}' на позиции {r['position']} с уверенностью {r['confidence']:.2f}")

print("\nРезультаты для запроса 'страна':")
for r in results2:
    print(f"Найдено '{r['token']}' в документе '{r['document']}' на позиции {r['position']} с уверенностью {r['confidence']:.2f}")


ValueError: Размерность эмбеддингов (1024) не соответствует размерности индекса (768)

In [42]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    Doc,
    NamesExtractor,
    PER
)
import re

class TextPreprocessor:
    """
    Класс для лингвистической предобработки русскоязычных текстов с использованием Natasha
    """
    
    def __init__(self):
        # Инициализация компонентов Natasha
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)
    
    def _clean_text(self, text):
        """
        Предобработка текста: нижний регистр, удаление пунктуации, нормализация пробелов
        """
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
        text = ' '.join(text.split())  # Нормализация пробелов
        return text
    
    def process(self, text):
        """
        Основной метод обработки текста
        
        Возвращает структуру:
        {
            "clean_text": str,
            "sentences": [
                {
                    "text": str,
                    "tokens": [
                        {
                            "text": str,
                            "lemma": str,
                            "pos": str,
                            "start": int,
                            "stop": int
                        }
                    ]
                }
            ],
            "entities": [
                {
                    "text": str,
                    "normal": str,
                    "type": str,
                    "start": int,
                    "stop": int
                }
            ]
        }
        """
        # Шаг 1: Предобработка
        cleaned_text = self._clean_text(text)
        doc = Doc(cleaned_text)
        
        # Шаг 2: Сегментация на предложения
        doc.segment(self.segmenter)
        
        # Шаг 3: Токенизация (выполняется автоматически при сегментации)
        
        # Шаг 4: Морфологический анализ
        doc.tag_morph(self.morph_tagger)
        
        # Шаг 5: Лемматизация
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        # Шаг 6: Извлечение сущностей
        doc.tag_ner(self.ner_tagger)
        
        # Шаг 7: Нормализация сущностей
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == PER:
                span.extract_fact(self.names_extractor)
        
        # Формирование выходной структуры
        return self._build_output(doc, cleaned_text)
    
    def _build_output(self, doc, cleaned_text):
        """Формирует структуру данных для LLM"""
        output = {
            "clean_text": cleaned_text,
            "sentences": [],
            "entities": []
        }
        
        # Обработка предложений
        for sent in doc.sents:
            sentence_data = {
                "text": sent.text,
                "tokens": []
            }
            
            for token in sent.tokens:
                token_data = {
                    "text": token.text,
                    "lemma": token.lemma,
                    "pos": token.pos,
                    "start": token.start,
                    "stop": token.stop
                }
                sentence_data["tokens"].append(token_data)
            
            output["sentences"].append(sentence_data)
        
        # Обработка сущностей
        for span in doc.spans:
            entity_data = {
                "text": span.text,
                "normal": span.normal,
                "type": span.type,
                "start": span.start,
                "stop": span.stop
            }
            output["entities"].append(entity_data)
        
        return output

# Пример использования
if __name__ == "__main__":
    preprocessor = TextPreprocessor()
    # sample_text = "Москва — столица России. Владимир Путин посетил завод в Подмосковье."
    sample_text = "Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈"
    
    result = preprocessor.process(sample_text)
    print("Очищенный текст:", result["clean_text"])
    print("\nТокены первого предложения:")
    for token in result["sentences"][0]["tokens"]:
        print(f"{token['text']} -> {token['lemma']} ({token['pos']})")
    
    print("\nИзвлеченные сущности:")
    for entity in result["entities"]:
        print(f"{entity['text']} -> {entity['normal']} ({entity['type']})")


Очищенный текст: твой лучший секс спрятан здесь делюсь каналом дипломированного сексолога крис взломала код классного секса мастерски раскрепощает знает миллион горячих техник и лучшие девайсы для взрослых самые полезные посты здесь отрезвляющий пост я все сама прокачай наездницу ролевая игра vip кинотеатр техника оральных ласк как занимается сeксом неудобная женщина кстати крис провела трехдневный безоплатный онлайн интенсивот бревна до богини совместно с врачом и владельцем сексшопа скорее смотри записи пока не удалила httpstmesekretskris1048 здесь жарче чем в аду

Токены первого предложения:
твой -> твой (DET)
лучший -> хороший (ADJ)
секс -> секс (NOUN)
спрятан -> спрятать (VERB)
здесь -> здесь (ADV)
делюсь -> делиться (VERB)
каналом -> канал (NOUN)
дипломированного -> дипломированный (ADJ)
сексолога -> сексолог (ADJ)
крис -> крис (NOUN)
взломала -> взломать (VERB)
код -> код (NOUN)
классного -> классный (ADJ)
секса -> секс (NOUN)
мастерски -> мастерски (ADV)
раскрепощает -> раскреп

In [45]:
from natasha import (Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, Doc)
import re
from typing import Dict, List, Any

class NatashaPreprocessor:
    """
    Класс для предобработки текста с использованием библиотеки Natasha.
    
    Выполняет предобработку русского текста для семантического поиска,
    включая приведение к нижнему регистру, удаление пунктуации, 
    морфологический анализ и лемматизацию.
    
    Результатом является структура данных, которую можно передать 
    языковой модели для создания эмбеддингов.
    """

    def __init__(self):
        """
        Инициализация предобработчика текста.
        Создает необходимые компоненты библиотеки Natasha.
        """
        self.segmenter = Segmenter()
        self.embedding = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.embedding)
        self.syntax_parser = NewsSyntaxParser(self.embedding)
        self.ner_tagger = NewsNERTagger(self.embedding)

    def preprocess(self, text: str) -> Dict[str, Any]:
        """
        Выполняет полную предобработку текста.
        
        Процесс обработки включает:
        1. Предобработка (нижний регистр, удаление пунктуации)
        2. Сегментация (разделение на предложения)
        3. Токенизация (разделение на слова)
        4. Морфоанализ (части речи, граммемы)
        5. Лемматизация (нормальная форма)
        6. NER (извлечение именованных сущностей)
        7. Нормализация сущностей
        
        Args:
            text (str): Исходный текст для обработки.
            
        Returns:
            Dict[str, Any]: Структура данных с обработанным текстом.
            
        Raises:
            ValueError: Если длина документа превышает 30 слов.
        """
        # Проверка длины документа
        words = text.split()
        if len(words) > 30:
            raise ValueError("Длина документа превышает максимально допустимые 30 слов")
        
        # 1. Предобработка: перевод в нижний регистр, удаление пунктуации
        text = text.lower()
        text = re.sub(r'[\W_]+', ' ', text)  # удаляем пунктуацию и спецсимволы

        # Создаем объект Doc для Natasha
        doc = Doc(text)

        # 2. Сегментация
        doc.segment(self.segmenter)

        # 3. Токенизация
        doc.tokens

        # 4. Морфоанализ
        doc.tag_morph(self.morph_tagger)

        # 5. Лемматизация
        for token in doc.tokens:
            token.lemmatize()

        # 6. NER
        doc.tag_ner(self.ner_tagger)

        # 7. Нормализация сущностей (приведение именованных сущностей к нормальной форме)
        entities = []
        for span in doc.spans:
            span.normalize(self.morph_tagger)
            entities.append({'text': span.normal, 'type': span.type, 'start': span.start, 'stop': span.stop})

        # Формируем структуру для передачи LLM (список лемм и сущностей с позициями)
        lemmas = [token.lemma for token in doc.tokens]
        
        # Добавляем информацию о позициях токенов и словосочетаниях
        tokens_info = []
        for token in doc.tokens:
            tokens_info.append({
                'text': token.text,
                'lemma': token.lemma,
                'start': token.start,
                'stop': token.stop
            })
        
        # Формируем словосочетания (до 2 слов)
        phrases = []
        # Одиночные слова
        for token in tokens_info:
            phrases.append({
                'text': token['text'],
                'lemma': token['lemma'],
                'start': token['start'],
                'stop': token['stop'],
                'length': 1
            })
        
        # Словосочетания из 2 слов
        for i in range(len(tokens_info) - 1):
            phrases.append({
                'text': f"{tokens_info[i]['text']} {tokens_info[i+1]['text']}",
                'lemma': f"{tokens_info[i]['lemma']} {tokens_info[i+1]['lemma']}",
                'start': tokens_info[i]['start'],
                'stop': tokens_info[i+1]['stop'],
                'length': 2
            })

        result = {
            'lemmas': lemmas,
            'entities': entities,
            'tokens': tokens_info,
            'phrases': phrases
        }

        return result


In [49]:
x = NatashaPreprocessor()

In [50]:
x.preprocess("Привет, как дела?")

TypeError: DocToken.lemmatize() missing 1 required positional argument: 'vocab'

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    Doc,
    NamesExtractor,
    NewsSyntaxParser,
    PER,
    LOC,
    ORG
)
import re

class TextPreprocessor:
    """
    Полнофункциональный препроцессор текста для семантического поиска
    с соблюдением требований хакатона.
    """
    
    def __init__(self):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)
        self.syntax_parser = NewsSyntaxParser(self.emb)
    
    def _clean_text(self, text):
        """Нормализация с сохранением позиций для дефисов/пунктуации"""
        cleaned = []
        original_to_clean = []
        clean_pos = 0

        for orig_pos, char in enumerate(text):
            if char.isalnum():
                cleaned.append(char.lower())
                original_to_clean.append(orig_pos)
                clean_pos += 1
            elif char.isspace():
                cleaned.append(' ')
                original_to_clean.append(orig_pos)
                clean_pos += 1
            else:
                # Сохраняем позиции для пунктуации/дефисов, но не включаем в очищенный текст
                original_to_clean.append(None)

        return ''.join(cleaned), original_to_clean

    
    def _get_original_positions(self, start, stop, mapping):
        """Преобразует позиции из очищенного текста в исходный"""
        orig_start = None
        orig_stop = None
        
        for i, val in enumerate(mapping):
            if val == start:
                orig_start = i
            if val == stop-1:
                orig_stop = i+1
            if orig_start is not None and orig_stop is not None:
                break
                
        return (orig_start, orig_stop) if orig_start is not None and orig_stop is not None else (start, stop)

    def process(self, text):
        """Основной метод обработки с ограничениями"""
        # Проверка длины документа
        words = text.split()
        if len(words) > 30:
            raise ValueError("Документ превышает максимальную длину в 30 слов")
        
        # Предобработка с сохранением позиций
        cleaned_text, pos_mapping = self._clean_text(text)
        doc = Doc(cleaned_text)
        
        # Обработка через Natasha
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        doc.parse_syntax(NewsSyntaxParser(self.emb))

        doc.tag_ner(self.ner_tagger)
        
        for span in doc.spans:
            # Нормализуем ВСЕ сущности, а не только PER
            span.normalize(self.morph_vocab)
            
            # Добавляем обработку для всех типов
            if span.type in [PER, LOC, ORG]:
                if span.type == PER:
                    span.extract_fact(self.names_extractor)
                # Добавляем сущность в результат
                entities.append({
                    "text": orig_text[start:stop],
                    "lemma": span.normal,
                    "type": span.type,  # <- Исправлено: было span.type
                    "start": start,
                    "stop": stop,
                    "length": len(span.tokens)
                })
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == PER:
                span.extract_fact(self.names_extractor)
        
        # Формирование структуры
        return self._build_output(doc, text, pos_mapping)

    def _build_output(self, doc, orig_text, pos_mapping):
        """Создает финальную структуру данных"""
        # Собираем все токены
        tokens_info = []
        for token in doc.tokens:
            orig_start, orig_stop = self._get_original_positions(
                token.start, token.stop, pos_mapping
            )
            tokens_info.append({
                "text": orig_text[orig_start:orig_stop],
                "lemma": token.lemma,
                "pos": token.pos,
                "start": orig_start,
                "stop": orig_stop
            })
        
        # Формируем словосочетания (1-2 слова)
        phrases = []
        used_spans = set()
        
        # Одиночные слова
        for token in tokens_info:
            phrases.append({
                "type": "word",
                "text": token["text"],
                "lemma": token["lemma"],
                "start": token["start"],
                "stop": token["stop"],
                "length": 1
            })
        
        # Пары слов
        for i in range(len(tokens_info)-1):
            phrase = {
                "type": "phrase",
                "text": f"{tokens_info[i]['text']} {tokens_info[i+1]['text']}",
                "lemma": f"{tokens_info[i]['lemma']} {tokens_info[i+1]['lemma']}",
                "start": tokens_info[i]['start'],
                "stop": tokens_info[i+1]['stop'],
                "length": 2
            }
            phrases.append(phrase)
        
        # Сущности из NER
        entities = []
        for span in doc.spans:
            start, stop = self._get_original_positions(span.start, span.stop, pos_mapping)
            entities.append({
                "text": orig_text[start:stop],
                "lemma": span.normal,
                "type": span.type,
                "start": start,
                "stop": stop,
                "length": len(span.tokens)
            })
        
        return {
            "lemmas": [token["lemma"] for token in tokens_info],
            "tokens": tokens_info,
            "phrases": phrases,
            "entities": entities,
            "original_text": orig_text,
            "clean_text": doc.text
        }

# Пример использования
if __name__ == "__main__":
    processor = TextPreprocessor()
    text = "Москва - столица России. Владимир Путин провел совещание."
    
    try:
        result = processor.process(text)
        print("Леммы:", result["lemmas"])
        print("\nСущности:")
        for ent in result["entities"]:
            print(f"{ent['text']} ({ent['type']}): {ent['lemma']}")
        
        print("\nСловосочетания:")
        for phrase in result["phrases"]:
            if phrase["length"] == 2:
                print(f"{phrase['text']} -> {phrase['lemma']}")
    except ValueError as e:
        print(e)


Леммы: ['москва', 'столица', 'россия', 'владимир', 'путин', 'провести', 'совещание']

Сущности:

Словосочетания:
Москва  столиц -> москва столица
 столиц  Росси -> столица россия
 Росси . Владим -> россия владимир
. Владим р Пут -> владимир путин
р Пут н пров -> путин провести
н пров л совещан -> провести совещание


In [55]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    NamesExtractor
)

class TextPreprocessor:
    """
    Класс для предобработки текста с учетом требований хакатона:
    - Ограничение длины документа (30 слов)
    - Извлечение сущностей (PER, LOC, ORG)
    - Формирование словосочетаний (1-2 слова)
    """
    
    def __init__(self):
        # Инициализация компонентов Natasha
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)  # Убрал параметр labels
        self.names_extractor = NamesExtractor(self.morph_vocab)

    def _clean_text(self, text):
        """
        Нормализация текста с сохранением позиций:
        - Приведение к нижнему регистру
        - Удаление пунктуации (кроме пробелов)
        - Возвращает очищенный текст и маппинг позиций
        """
        cleaned = []
        original_to_clean = []
        clean_pos = 0

        for orig_pos, char in enumerate(text):
            if char.isalnum() or char.isspace():
                cleaned.append(char.lower() if char.isalnum() else ' ')
                original_to_clean.append(orig_pos)
                clean_pos += 1
            else:
                original_to_clean.append(None)

        return ''.join(cleaned), original_to_clean

    def _get_original_positions(self, start, stop, mapping):
        """
        Преобразует позиции из очищенного текста в исходный
        с учетом удаленных символов
        """
        try:
            # Ищем первую не-None позицию в диапазоне
            orig_start = next(mapping[i] for i in range(start, len(mapping)) if mapping[i] is not None)
            # Ищем последнюю не-None позицию
            orig_stop = next(mapping[i] for i in reversed(range(stop)) if mapping[i] is not None) + 1
            return (orig_start, orig_stop)
        except StopIteration:
            return (start, stop)

    def process(self, text):
        """
        Основной метод обработки текста:
        - Проверяет длину документа
        - Выполняет полный цикл обработки через Natasha
        - Возвращает структурированные данные
        """
        # Проверка длины документа
        words = text.split()
        if len(words) > 30:
            raise ValueError("Документ превышает максимальную длину в 30 слов")
        
        # Предобработка текста
        cleaned_text, pos_mapping = self._clean_text(text)
        doc = Doc(cleaned_text)
        
        # Обработка через Natasha
        doc.segment(self.segmenter)       # Сегментация на предложения
        doc.tag_morph(self.morph_tagger)  # Морфологический анализ
        doc.parse_syntax(self.syntax_parser)  # Синтаксический анализ (ВАЖНО для NER)
        doc.tag_ner(self.ner_tagger)      # Извлечение сущностей
        
        # Лемматизация токенов
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        # Нормализация сущностей
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == PER:
                span.extract_fact(self.names_extractor)
        
        # Формирование выходной структуры
        return self._build_output(doc, text, pos_mapping)

    def _build_output(self, doc, orig_text, pos_mapping):
        """
        Формирует итоговую структуру данных:
        - Леммы
        - Токены с позициями
        - Словосочетания
        - Сущности
        """
        # Сбор информации о токенах
        tokens_info = []
        for token in doc.tokens:
            orig_start, orig_stop = self._get_original_positions(token.start, token.stop, pos_mapping)
            tokens_info.append({
                "text": orig_text[orig_start:orig_stop],
                "lemma": token.lemma,
                "pos": token.pos,
                "start": orig_start,
                "stop": orig_stop
            })
        
        # Формирование словосочетаний (1-2 слова)
        phrases = []
        
        # 1. Одиночные слова
        for token in tokens_info:
            phrases.append({
                "type": "word",
                "text": token["text"],
                "lemma": token["lemma"],
                "start": token["start"],
                "stop": token["stop"],
                "length": 1
            })
        
        # 2. Пары слов (только внутри одного предложения)
        current_sentence_end = 0
        for sent in doc.sents:
            sentence_tokens = [t for t in tokens_info if t["start"] >= current_sentence_end]
            if sentence_tokens:
                current_sentence_end = sentence_tokens[-1]["stop"]
                
                # Генерируем пары только внутри предложения
                for i in range(len(sentence_tokens) - 1):
                    token1 = sentence_tokens[i]
                    token2 = sentence_tokens[i+1]
                    phrases.append({
                        "type": "phrase",
                        "text": f"{token1['text']} {token2['text']}",
                        "lemma": f"{token1['lemma']} {token2['lemma']}",
                        "start": token1["start"],
                        "stop": token2["stop"],
                        "length": 2
                    })
        
        # Извлечение сущностей
        entities = []
        for span in doc.spans:
            start, stop = self._get_original_positions(span.start, span.stop, pos_mapping)
            
            # Сравниваем с названиями типов в виде строк
            if span.type in ['PER', 'LOC', 'ORG']:
                entities.append({
                    "text": orig_text[start:stop],
                    "lemma": span.normal,
                    "type": span.type,  # тип уже является строкой
                    "start": start,
                    "stop": stop,
                    "length": len(span.tokens)
                })
        
        return {
            "lemmas": [token["lemma"] for token in tokens_info],
            "tokens": tokens_info,
            "phrases": phrases,
            "entities": entities,
            "original_text": orig_text,
            "clean_text": doc.text
        }

# Пример использования
if __name__ == "__main__":
    processor = TextPreprocessor()
    sample_text = "Москва — столица России. Президент Владимир Путин провел совещание."
    
    try:
        result = processor.process(sample_text)
        print("Леммы:", result["lemmas"])
        print("\nСущности:")
        for ent in result["entities"]:
            print(f"{ent['text']} ({ent['type']}): {ent['lemma']}")
        
        print("\nСловосочетания:")
        for phrase in result["phrases"]:
            if phrase["length"] == 2:
                print(f"{phrase['text']} -> {phrase['lemma']}")
    except ValueError as e:
        print(e)


Леммы: ['москва', 'столица', 'россия', 'президент', 'владимир', 'путин', 'провести', 'совещание']

Сущности:

Словосочетания:
Москва  столиц -> москва столица
 столиц  Росси -> столица россия
 Росси  Президе -> россия президент
 Президе т Владим -> президент владимир
т Владим р Пут -> владимир путин
р Пут н пров -> путин провести
н пров л совещан -> провести совещание


In [57]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    NamesExtractor
)
import re

class TextPreprocessor:
    def __init__(self):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)  # Исправлено: убран параметр labels
        self.names_extractor = NamesExtractor(self.morph_vocab)

    def _clean_text(self, text):
        cleaned = []
        original_to_clean = []
        clean_pos = 0
        for orig_pos, char in enumerate(text):
            if char.isalnum() or char.isspace():
                cleaned.append(char.lower() if char.isalnum() else ' ')
                original_to_clean.append(orig_pos)
                clean_pos += 1
            else:
                original_to_clean.append(None)
        return ''.join(cleaned), original_to_clean

    def _get_original_positions(self, start, stop, mapping):
        try:
            orig_start = mapping[start]
            orig_stop = mapping[stop-1] + 1
            return (orig_start, orig_stop)
        except (IndexError, StopIteration):
            return (start, stop)

    def process(self, text):
        if len(text.split()) > 30:
            raise ValueError("Документ превышает 30 слов")
        cleaned_text, pos_mapping = self._clean_text(text)
        doc = Doc(cleaned_text)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        doc.tag_ner(self.ner_tagger)
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == 'PER':
                span.extract_fact(self.names_extractor)
        return self._build_output(doc, text, pos_mapping)

    def _build_output(self, doc, orig_text, pos_mapping):
        tokens_info = []
        for token in doc.tokens:
            start, stop = self._get_original_positions(token.start, token.stop, pos_mapping)
            tokens_info.append({
                "text": orig_text[start:stop],
                "lemma": token.lemma,
                "pos": token.pos,
                "start": start,
                "stop": stop
            })
        phrases = []
        # Одиночные слова
        for token in tokens_info:
            phrases.append({
                "type": "word",
                "text": token["text"],
                "lemma": token["lemma"],
                "start": token["start"],
                "stop": token["stop"],
                "length": 1
            })
        # Биграммы
        for i in range(len(tokens_info)-1):
            phrases.append({
                "type": "phrase",
                "text": f"{tokens_info[i]['text']} {tokens_info[i+1]['text']}",
                "lemma": f"{tokens_info[i]['lemma']} {tokens_info[i+1]['lemma']}",
                "start": tokens_info[i]['start'],
                "stop": tokens_info[i+1]['stop'],
                "length": 2
            })
        entities = []
        for span in doc.spans:
            if span.type in ['PER', 'LOC', 'ORG']:  # Сравнение по строковым значениям
                start, stop = self._get_original_positions(span.start, span.stop, pos_mapping)
                entities.append({
                    "text": orig_text[start:stop],
                    "lemma": span.normal,
                    "type": span.type,
                    "start": start,
                    "stop": stop,
                    "length": len(span.tokens)
                })
        return {
            "lemmas": [token["lemma"] for token in tokens_info],
            "tokens": tokens_info,
            "phrases": phrases,
            "entities": entities,
            "original_text": orig_text,
            "clean_text": doc.text
        }

# Пример использования
if __name__ == "__main__":
    processor = TextPreprocessor()
    text = "Москва — столица России. Владимир Путин провел совещание."
    try:
        result = processor.process(text)
        print(result)
    except ValueError as e:
        print(e)


{'lemmas': ['москва', 'столица', 'россия', 'владимир', 'путин', 'провести', 'совещание'], 'tokens': [{'text': 'Москва', 'lemma': 'москва', 'pos': 'VERB', 'start': 0, 'stop': 6}, {'text': ' столиц', 'lemma': 'столица', 'pos': 'NOUN', 'start': 8, 'stop': 15}, {'text': ' Росси', 'lemma': 'россия', 'pos': 'NOUN', 'start': 16, 'stop': 22}, {'text': 'Москва — столица России. Владим', 'lemma': 'владимир', 'pos': 'NOUN', 'start': None, 'stop': 31}, {'text': 'р Пут', 'lemma': 'путин', 'pos': 'PROPN', 'start': 32, 'stop': 37}, {'text': 'н пров', 'lemma': 'провести', 'pos': 'VERB', 'start': 38, 'stop': 44}, {'text': 'л совещан', 'lemma': 'совещание', 'pos': 'NOUN', 'start': 45, 'stop': 54}], 'phrases': [{'type': 'word', 'text': 'Москва', 'lemma': 'москва', 'start': 0, 'stop': 6, 'length': 1}, {'type': 'word', 'text': ' столиц', 'lemma': 'столица', 'start': 8, 'stop': 15, 'length': 1}, {'type': 'word', 'text': ' Росси', 'lemma': 'россия', 'start': 16, 'stop': 22, 'length': 1}, {'type': 'word', 'te

In [ ]:
result

In [58]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    NamesExtractor
)
import re

class TextPreprocessor:
    def __init__(self):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)  # Добавлен синтаксический анализ
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)

    def _clean_text(self, text):
        cleaned = []
        original_to_clean = []
        for orig_pos, char in enumerate(text):
            if char.isalnum() or char.isspace():
                cleaned.append(char.lower() if char.isalnum() else ' ')
                original_to_clean.append(orig_pos)
            else:
                original_to_clean.append(None)
        return ''.join(cleaned), original_to_clean

    def _get_original_positions(self, start, stop, mapping):
        try:
            orig_start = next(i for i in range(start, len(mapping)) if mapping[i] is not None)
            orig_stop = next(i for i in reversed(range(stop, len(mapping))) if mapping[i] is not None) + 1
            return (orig_start, orig_stop)
        except StopIteration:
            return (start, stop)

    def process(self, text):
        if len(text.split()) > 30:
            raise ValueError("Документ превышает 30 слов")
        
        cleaned_text, pos_mapping = self._clean_text(text)
        doc = Doc(cleaned_text)
        
        # Обязательные этапы обработки
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)  # Критически важно для NER
        doc.tag_ner(self.ner_tagger)
        
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        # Нормализация сущностей
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == 'PER':
                span.extract_fact(self.names_extractor)
        
        return self._build_output(doc, text, pos_mapping)

    def _build_output(self, doc, orig_text, pos_mapping):
        tokens_info = []
        for token in doc.tokens:
            start, stop = self._get_original_positions(token.start, token.stop, pos_mapping)
            tokens_info.append({
                "text": orig_text[start:stop],
                "lemma": token.lemma,
                "pos": token.pos,
                "start": start,
                "stop": stop
            })
        
        # Формирование словосочетаний
        phrases = []
        for i in range(len(tokens_info)):
            phrases.append({
                "text": tokens_info[i]['text'],
                "lemma": tokens_info[i]['lemma'],
                "start": tokens_info[i]['start'],
                "stop": tokens_info[i]['stop'],
                "length": 1
            })
            if i < len(tokens_info)-1:
                phrases.append({
                    "text": f"{tokens_info[i]['text']} {tokens_info[i+1]['text']}",
                    "lemma": f"{tokens_info[i]['lemma']} {tokens_info[i+1]['lemma']}",
                    "start": tokens_info[i]['start'],
                    "stop": tokens_info[i+1]['stop'],
                    "length": 2
                })
        
        # Извлечение сущностей
        entities = []
        for span in doc.spans:
            if span.type in ['PER', 'LOC', 'ORG']:
                start, stop = self._get_original_positions(span.start, span.stop, pos_mapping)
                entities.append({
                    "text": orig_text[start:stop],
                    "lemma": span.normal,
                    "type": span.type,
                    "start": start,
                    "stop": stop,
                    "length": len(span.tokens)
                })
        
        return {
            "lemmas": [t['lemma'] for t in tokens_info],
            "tokens": tokens_info,
            "phrases": phrases,
            "entities": entities,
            "original_text": orig_text,
            "clean_text": doc.text
        }

# Пример использования с проверкой
if __name__ == "__main__":
    processor = TextPreprocessor()
    text = "Москва — столица России. Владимир Путин провел совещание."
    
    try:
        result = processor.process(text)
        print("Сущности:", [(ent["text"], ent["type"]) for ent in result["entities"]])
        print("Леммы:", result["lemmas"])
    except Exception as e:
        print(f"Ошибка: {str(e)}")


Сущности: []
Леммы: ['москва', 'столица', 'россия', 'владимир', 'путин', 'провести', 'совещание']


In [59]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    NamesExtractor
)
import re

class TextPreprocessor:
    def __init__(self):
        # Инициализация компонентов Natasha
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)  # Исправлено: убран параметр labels
        self.names_extractor = NamesExtractor(self.morph_vocab)

    def _clean_text(self, text):
        """Нормализация текста с сохранением позиций"""
        cleaned = []
        original_to_clean = []
        for orig_pos, char in enumerate(text):
            if char.isalnum() or char.isspace():
                cleaned.append(char.lower() if char.isalnum() else ' ')
                original_to_clean.append(orig_pos)
            else:
                original_to_clean.append(None)
        return ''.join(cleaned), original_to_clean

    def _get_original_positions(self, start, stop, mapping):
        """Корректный маппинг позиций с учетом удаленных символов"""
        try:
            orig_start = next(i for i in range(start, len(mapping)) if mapping[i] is not None)
            orig_stop = next(i for i in reversed(range(stop)) if mapping[i] is not None) + 1
            return (orig_start, orig_stop)
        except StopIteration:
            return (start, stop)

    def process(self, text):
        """Основной метод обработки текста"""
        # Проверка длины документа
        if len(text.split()) > 30:
            raise ValueError("Документ превышает 30 слов")
        
        # Предобработка текста
        cleaned_text, pos_mapping = self._clean_text(text)
        doc = Doc(cleaned_text)
        
        # Полный цикл обработки
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)  # Обязательно для NER
        doc.tag_ner(self.ner_tagger)  # Критически важный шаг
        
        # Лемматизация
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        # Обработка сущностей
        for span in doc.spans:
            span.normalize(self.morph_vocab)
            if span.type == 'PER':  # Сравнение по строке
                span.extract_fact(self.names_extractor)
        
        return self._build_output(doc, text, pos_mapping)

    def _build_output(self, doc, orig_text, pos_mapping):
        """Формирование итоговой структуры"""
        # Токены
        tokens_info = []
        for token in doc.tokens:
            start, stop = self._get_original_positions(token.start, token.stop, pos_mapping)
            tokens_info.append({
                "text": orig_text[start:stop],
                "lemma": token.lemma,
                "pos": token.pos,
                "start": start,
                "stop": stop
            })
        
        # Словосочетания
        phrases = []
        for i in range(len(tokens_info)):
            # Одиночные слова
            phrases.append({
                "text": tokens_info[i]['text'],
                "lemma": tokens_info[i]['lemma'],
                "start": tokens_info[i]['start'],
                "stop": tokens_info[i]['stop'],
                "length": 1
            })
            # Биграммы
            if i < len(tokens_info)-1:
                phrases.append({
                    "text": f"{tokens_info[i]['text']} {tokens_info[i+1]['text']}",
                    "lemma": f"{tokens_info[i]['lemma']} {tokens_info[i+1]['lemma']}",
                    "start": tokens_info[i]['start'],
                    "stop": tokens_info[i+1]['stop'],
                    "length": 2
                })
        
        # Сущности
        entities = []
        for span in doc.spans:
            if span.type in ['PER', 'LOC', 'ORG']:  # Фильтрация по строковым типам
                start, stop = self._get_original_positions(span.start, span.stop, pos_mapping)
                entities.append({
                    "text": orig_text[start:stop],
                    "lemma": span.normal,
                    "type": span.type,
                    "start": start,
                    "stop": stop,
                    "length": len(span.tokens)
                })
        
        return {
            "lemmas": [t['lemma'] for t in tokens_info],
            "tokens": tokens_info,
            "phrases": phrases,
            "entities": entities,
            "original_text": orig_text,
            "clean_text": doc.text
        }

# Пример использования
if __name__ == "__main__":
    processor = TextPreprocessor()
    text = "Москва — столица России. Владимир Путин провел совещание."
    
    try:
        result = processor.process(text)
        print("Сущности:", [(ent["text"], ent["type"]) for ent in result["entities"]])
        print("Леммы:", result["lemmas"])
    except Exception as e:
        print(f"Ошибка: {str(e)}")


Сущности: []
Леммы: ['москва', 'столица', 'россия', 'владимир', 'путин', 'провести', 'совещание']


In [60]:
!pip info natasha

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
19823.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ERROR: unknown command "info"
